In [1]:
import numpy as np
np.random.seed(42)

In [2]:
class NeuralNetwork:

  def __init__(self):
    self.W = {}
    self.b = {}
    self.dW = {}
    self.db = {}
    self.layer_infos = []
  
  def add_layer(self, n_neurons, activation, n_inputs = 0):
    if n_inputs == 0:
      last_key = list(self.W)[-1]
      layer_weights =  np.random.randn(n_neurons,self.W[last_key].shape[0])
      layer_biases = np.zeros((1, n_neurons))
      self.W["W" + str(int(last_key[-1]) + 1)] = layer_weights
      self.b["b" + str(int(last_key[-1]) + 1)] = layer_biases
    else:
      layer_weights = 0.10 * np.random.randn(n_neurons,n_inputs)
      layer_biases = np.zeros((1, n_neurons))
      self.W["W" + str(1)] = layer_weights
      self.b["b" + str(1)] = layer_biases

    self.layer_infos.append(activation)

  def fit(self, X, y, epochs, learning_rate):
    cost_history = []
    accuracy_history = []

    for _ in range(epochs):
      y_pred, memory = self._forward(X)
      cost = self._cost(y_pred, y)
      cost_history.append(cost)
      accuracy = self._accuracy(y_pred, y)
      accuracy_history.append(accuracy)
  

      self._backward(y_pred, y, memory)

      self._update(learning_rate)

      print("accuracy : " + str(accuracy) + ", lost : " + str(cost) )

    return cost_history, accuracy_history

  def _forward(self, X):
    memory = {}
    A_current = X

    for i in range(len(self.layer_infos)):
      layer_index = i + 1

      A_previous = A_current

      if self.layer_infos[i] == "sigmoid":
        X_current = np.dot(A_previous, self.W["W" + str(layer_index)].T) + self.b["b" + str(layer_index)]
        A_current = self._sigmoid(X_current)
        memory["X" + str(layer_index)] = X_current
      elif self.layer_infos[i] == "relu":
        X_current = np.dot(A_previous, self.W["W" + str(layer_index)].T) +  self.b["b" + str(layer_index)]
        A_current = self._relu(X_current)
        memory["X" + str(layer_index)] = X_current
      
      memory["A" + str(i)] = A_previous
      
    return A_current, memory
  
  def _backward(self, y_pred, y, memory):
    y = y.reshape(y_pred.shape)
    epsilon = 1e-5

    dA_previous = - (np.divide(y, y_pred + epsilon) - np.divide( 1 - y, 1 - y_pred + epsilon))

    for i in reversed(range(len(self.layer_infos))):
      m = memory["A" + str(i)].shape[1]
      layer_index_current = i + 1


      dA_current = dA_previous

      if self.layer_infos[i] == "sigmoid":
        dX_current = self._sigmoid_derivative(dA_current, memory["X" + str(layer_index_current)])
      if self.layer_infos[i] == "relu":
        dX_current = self._relu_derivative(dA_current, memory["X" + str(layer_index_current)])

      dW_current = np.dot(dX_current.T, memory["A" + str(i)]) / m
      db_current = np.sum(dX_current, axis = 0, keepdims = True) / m
      dA_previous = np.dot(dX_current,self.W["W" + str(layer_index_current)])

    
      self.dW["dW" + str(layer_index_current)] = dW_current
      self.db["db" + str(layer_index_current)] = db_current

  def _update(self,learning_rate):
    for i in range(len(self.layer_infos)):
      self.W["W" + str(i+1)] += learning_rate * self.dW["dW" + str(i+1)]
      self.b["b" + str(i+1)] += learning_rate * self.db["db" + str(i+1)]

  def _cost(self, y_pred, y):
    m = y_pred.shape[1]
    epsilon = 1e-5
    cost = (-1/m) * (np.dot(y, np.log(y_pred + epsilon).T) + np.dot(1-y, np.log(1 - y_pred+epsilon).T))
    return np.sum(cost)

  def _accuracy(self, y_pred, y):
    m = y_pred.shape[0]
    y_pred[y_pred > 0.5] = 1
    y_pred[y_pred <= 0.5] = 0
    accuracy = np.sum(y == y_pred) / m
    return accuracy

  def _sigmoid(self, X):
    return 1 / ( 1 + np.exp(-X))

  def _relu(self, X):
    return np.maximum(0, X)

  def _sigmoid_derivative(self, dA, X):
    sig = self._sigmoid(X)
    return dA * sig * ( 1 - sig )

  def _relu_derivative(self, dA, X):
    dX = np.array(dA, copy = True)
    dX[X <= 0] = 0
    return dX



In [3]:
model = NeuralNetwork()

model.add_layer(20, activation = "relu", n_inputs = 4096)
model.add_layer(20, activation = "relu")

model.add_layer(1, activation = "sigmoid")

In [4]:
data = np.load("/content/drive/MyDrive/Colab Notebooks/data.npy")

np.random.shuffle(data)

slicer = int(data.shape[0]*0.9)
X_train = data[:slicer,:-1]
X_test = data[slicer:data.shape[0],:-1]
y_train = data[:slicer,-1]
y_train = y_train.reshape(len(y_train),1)
y_test = data[slicer:data.shape[0],-1]
y_test = y_test.reshape(len(y_test),1)

In [5]:
cost, accuracy = model.fit(X_train,y_train, 3 , 0.01)

accuracy : 0.48739495798319327, lost : 5326243.297012291
accuracy : 0.48739495798319327, lost : 5326243.297012291
accuracy : 0.48739495798319327, lost : 5326243.297012291
